In [1]:
import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import random
import math


access_token = "e11aadc917842adf254cd73c038c4e0a-321ea21ac5697ab46036807f5e5e943d"

funds_data = {
    'SPX500_USD': {},
    'JP225_USD': {},
    'DE30_EUR': {}, 
    'US2000_USD': {},
    'NAS100_USD': {},
    'FR40_EUR': {},
    'CN50_USD': {},
    'HK33_HKD': {},
    'AU200_AUD': {},   
    'UK100_GBP': {}, 

    
    # Add more index funds as needed
}

client = API(access_token=access_token)

for fund in funds_data:
    params = {
        "count": 5000,
        "granularity": "H1"
    }
    r = instruments.InstrumentsCandles(instrument=fund, params=params)
    client.request(r)
    response = r.response

    candles = response["candles"]
    prices = [float(candle["mid"]["c"]) for candle in candles]
    volumes = [int(candle["volume"]) for candle in candles]
    funds_data[fund]['Close'] = prices
    funds_data[fund]['Volume'] = volumes

class State:
    def __init__(self, funds):
        self.funds = funds
        self.volume = {fund: data['Volume'] for fund, data in funds.items()}

class Action:
    def __init__(self, fund):
        self.fund = fund

class Node:
    def __init__(self, state, action=None, parent=None):
        self.state = state
        self.action = action
        self.parent = parent
        self.children = []
        self.visits = 0
        self.reward = 0

def UCB(node, exploration_constant=1.414):
    if node.visits == 0:
        return float("inf")
    return (node.reward / node.visits) + exploration_constant * math.sqrt(math.log(node.parent.visits) / node.visits)

def expand(node):
    available_funds = list(node.state.funds.keys())
    for fund in available_funds:
        action = Action(fund)
        child_state = State(node.state.funds)
        child_node = Node(child_state, action, node)
        node.children.append(child_node)
    return node.children

def simulate(node, steps):
    fund = random.choice(list(node.state.funds.keys()))
    fund_data = node.state.funds[fund]
    prices = fund_data['Close']
    cumulative_growth = 1.0
    predictions = []
    for step in range(steps):
        step_predictions = []
        for i in range(len(prices) - 1):
            price_today = prices[i]
            price_tomorrow = prices[i + 1]
            daily_growth = price_tomorrow / price_today
            cumulative_growth *= daily_growth
            if daily_growth > 1.0:
                step_predictions.append(1)  # Predict growth
            else:
                step_predictions.append(-1)  # Predict decline
        predictions.append(step_predictions)
    return cumulative_growth * random.uniform(0.9, 1.1), predictions

def backpropagate(node, reward):
    node.visits += 1
    node.reward += reward
    if node.parent:
        backpropagate(node.parent, reward)

def MCTS(root, iterations, steps, growth_threshold=1.01, decline_threshold=0.99):
    for _ in range(iterations):
        node = root
        while node.children:
            node = max(node.children, key=UCB)
        if node.visits == 0:
            expand(node)
        reward, predictions = simulate(node, steps)
        backpropagate(node, reward)

    max_child = max(root.children, key=lambda n: n.visits)
    fund = max_child.action.fund
    if all(prediction == 1 for prediction in predictions[-1]):
        return f"The fund '{fund}' is expected to grow."
    elif all(prediction == -1 for prediction in predictions[-1]):
        return f"The fund '{fund}' is expected to decline."
    elif reward >= growth_threshold:
        return f"The fund '{fund}' is likely to grow."
    elif reward <= decline_threshold:
        return f"The fund '{fund}' is likely to decline."
    else:
        return None

def MCTS_ensemble(root, iterations, num_models, steps, growth_threshold=1.01, decline_threshold=0.99):
    predictions = []
    for i in range(num_models):
        exploration_constant = 1.414 * (i + 1)
        for _ in range(iterations):
            node = root
            while node.children:
                node = max(node.children, key=lambda n: UCB(n, exploration_constant))
            if node.visits == 0:
                expand(node)
            reward, _ = simulate(node, steps)
            backpropagate(node, reward)

        max_child = max(root.children, key=lambda n: n.visits)
        fund = max_child.action.fund
        if reward >= growth_threshold:
            predictions.append(1)  # Predict growth
        elif reward <= decline_threshold:
            predictions.append(-1)  # Predict decline
        else:
            predictions.append(0)  # No prediction

    growth_predictions = sum(1 for prediction in predictions if prediction == 1)
    decline_predictions = sum(1 for prediction in predictions if prediction == -1)
    if growth_predictions / num_models >= 0.6:
        return f"The fund '{fund}' is expected to grow."
    elif decline_predictions / num_models >= 0.6:
        return f"The fund '{fund}' is expected to decline."
    else:
        return None
    
initial_state = State(funds_data)
root = Node(initial_state)
prediction = MCTS_ensemble(root, iterations=2000, num_models=5, steps=6)
print(prediction)


The fund 'SPX500_USD' is expected to grow.
